In [ ]:
# Import the pandas library. It is widely used for data manipulation and analysis, 
# allowing you to work with structured data such as CSV files, Excel spreadsheets, etc.
import pandas as pd
# Import the NumPy library. NumPy provides support for large, multi - dimensional arrays and matrices, 
# along with a collection of mathematical functions to operate on these arrays.
import numpy as np
# Import the XGBoost library. XGBoost is a powerful gradient - boosting library 
# that can be used for regression and classification tasks.
import xgboost as xgb
# Import the Gradio library. Gradio makes it easy to create customizable web interfaces for machine learning models.
import gradio as gr
# Import the StandardScaler class from sklearn.preprocessing. 
# StandardScaler is used to standardize features by removing the mean and scaling to unit variance.
from sklearn.preprocessing import StandardScaler
# Import joblib, which is used for saving and loading Python objects, 
# especially useful for saving and loading machine learning models.
import joblib

# Load the shear force prediction model
# Initialize an XGBoost regression model for shear force
shear_model = xgb.XGBRegressor()
# Load the pre - trained shear force prediction model from the specified JSON file
shear_model.load_model("xgboost_shear.json")

# Load the drift prediction model
# Initialize an XGBoost regression model for drift
drift_model = xgb.XGBRegressor()
# Load the pre - trained drift prediction model from the specified JSON file
drift_model.load_model("xgboost_drift.json")

# Load the scaler
# Load the pre - fitted StandardScaler from the specified file. 
# This scaler was used during the model training to standardize the input data.
scaler = joblib.load('scaler.joblib')

# Define the prediction function
# This function takes in 8 input parameters and returns the predicted shear force and drift.
def predict(D, H_D, G, P, ED, PGV, Sa10, Sa20):
    # Create a 2D NumPy array with the input parameters for the model.
    # The input data needs to be in a format that the model can accept.
    input_data = np.array([[D, H_D, G, P, ED, PGV, Sa10, Sa20]])
    # Scale the input data using the previously loaded scaler.
    # Standardizing the input data ensures that it has the same scale as the data used for training the model.
    input_scaled = scaler.transform(input_data)
    # Predict the shear force using the shear model.
    # The model takes the scaled input data and returns a prediction array.
    pred_shear = shear_model.predict(input_scaled)
    # Extract the predicted shear force from the prediction array.
    # Since the prediction array contains only one element in this case, we take the first element.
    shear = pred_shear[0]
    # Predict the drift using the drift model.
    pred_drift = drift_model.predict(input_scaled)
    # Extract the predicted drift from the prediction array.
    drift = pred_drift[0]
    # Return both the predicted shear force and drift.
    return shear, drift

# Create a Gradio interface for user interaction
# Use gr.Blocks to create a custom - structured interface with a specified theme.
with gr.Blocks(theme=gr.themes.Default(
    # Set the primary color of the interface to blue.
    primary_hue="blue",
    # Set the secondary color of the interface to gray.
    secondary_hue="gray",
    # Set the neutral color of the interface to slate.
    neutral_hue="slate",
    # Set the font for the interface. Use Roboto as the primary font, 
    # and fall back to system - provided sans - serif fonts if Roboto is not available.
    font=[gr.themes.GoogleFont("Roboto"), "ui - sans - serif", "system - ui", "sans - serif"]
)) as iface:
    # Display a title at the top of the interface using Markdown.
    # The title is centered and has a specific color.
    gr.Markdown("<h1 style='text-align: center; color: #2c3e50;'>Seismic Response Prediction</h1>")
    # Display a description below the title using Markdown.
    # The description is centered and has a specific color, guiding the user on what to do.
    gr.Markdown("<p style='text-align: center; color: #7f8c8d;'>Input parameters for seismic response prediction, and click 'Submit' button below.</p>")

    # Display a subtitle for the input parameters section.
    gr.Markdown("<h3 style='color: #2c3e50;'>Input Parameters</h3>")
    # Use gr.Row to create a horizontal layout for the input parameters.
    with gr.Row():
        # First row of input parameters
        with gr.Row():
            # Create a number input field for the diameter with a default value and a label.
            D = gr.Number(label="Diameter (m)", value=2.2)
            # Create a number input field for the aspect ratio with a default value and a label.
            H_D = gr.Number(label="Aspect ratio", value=4.5)
        # Second row of input parameters
        with gr.Row():
            # Create a number input field for the gravity load ratio with a default value and a label.
            G = gr.Number(label="Gravity load ratio", value=0.059)
            # Create a number input field for the initial PT force ratio with a default value and a label.
            P = gr.Number(label="Initial PT force ratio", value=0.077)
        # Third row of input parameters
        with gr.Row():
            # Create a number input field for the longitudinal reinforcing steel ratio with a default value and a label.
            ED = gr.Number(label="Reinforcing steel ratio", value=0.00153)
            # Create a number input field for the PGV with a default value and a label.
            PGV = gr.Number(label="PGV (cm/s)", value=58.4)
        # Fourth row of input parameters
        with gr.Row():
            # Create a number input field for the Sa10 with a default value and a label.
            Sa10 = gr.Number(label="Sa10 (g)", value=0.78)
            # Create a number input field for the Sa20 with a default value and a label.
            Sa20 = gr.Number(label="Sa20 (g)", value=0.21)

    # Display a subtitle for the output response section.
    gr.Markdown("<h3 style='color: #2c3e50;'>Output Response</h3>")
    # Use gr.Row to create a horizontal layout for the output fields.
    with gr.Row():
        # Create a number output field for the predicted shear force with a label.
        shear_output = gr.Number(label="Shear (kN)")
        # Create a number output field for the predicted drift with a label.
        drift_output = gr.Number(label="Drift (%)")

    # Create a submit button with a primary style.
    submit_button = gr.Button("Submit", variant="primary")
    # Connect the submit button to the prediction function.
    # When the button is clicked, the predict function will be called with the input values, 
    # and the results will be displayed in the output fields.
    submit_button.click(
        fn=predict,
        inputs=[D, H_D, G, P, ED, PGV, Sa10, Sa20],
        outputs=[shear_output, drift_output]
    )

# Launch the Gradio interface for user interaction.
# This will start a web server and open the interface in a browser.
iface.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


c:\Program Files\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Program Files\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
